In [1]:
#dependiencies
import pandas as pd
from sodapy import Socrata
from key import token,username,password

In [2]:
#client to connect to cdc covid-19 data
client = Socrata("data.cdc.gov",
                  token,
                  username,
                  password)

In [3]:
# results from cdc
results = client.get("9mfq-cb36", limit=23000)

In [4]:
# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [5]:
results_df

,submission_date,state,tot_cases,new_case,pnew_case,tot_death,new_death,pnew_death,created_at,consent_cases,consent_deaths,conf_death,prob_death,conf_cases,prob_cases
0,2020-10-15T00:00:00.000,DC,16166,34.0,0,638,0.0,0,2020-10-16T13:33:55.808,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-12-08T00:00:00.000,NM,109947,0.0,0,1756,0.0,0,2020-12-09T14:45:40.234,N/A,Not agree,NaN,NaN,NaN,NaN
2,2021-01-01T00:00:00.000,FL,1300528,0.0,6063,21673,0.0,7,2021-01-02T14:50:51.219,Not agree,Not agree,NaN,NaN,NaN,NaN
3,2020-08-29T00:00:00.000,SD,12942,425.0,0,167,2.0,0,2020-08-30T14:49:52.418,N/A,Agree,165.0,2,NaN,NaN
4,2020-04-30T00:00:00.000,IA,7145,302.0,0,162,14.0,0,2020-05-01T21:00:19.025,Not agree,Not agree,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22795,2020-03-01T00:00:00.000,WY,0,0.0,NaN,0,0.0,NaN,2020-03-26T16:22:39.452,Agree,Agree,NaN,NaN,NaN,NaN
22796,2020-07-12T00:00:00.000,DC,10847,46.0,0,568,0.0,0,2020-07-13T15:19:34.972,NaN,NaN,NaN,NaN,NaN,NaN
22797,2020-03-05T00:00:00.000,MI,0,0.0,NaN,0,0.0,NaN,2020-03-26T16:22:39.452,Agree,Agree,NaN,NaN,NaN,NaN
22798,2021-02-03T00:00:00.000,CA,3281271,10501.0,0,41811,481.0,0,2021-02-04T14:53:58.929,Not agree,Not agree,NaN,NaN,NaN,NaN


In [6]:
#reformatting date
results_df['submission_date']=pd.to_datetime(results_df['submission_date']).dt.date
results_df['created_at']=pd.to_datetime(results_df['created_at']).dt.date

In [7]:
results_df['tot_cases']=results_df['tot_cases'].astype('float')
results_df['conf_cases']=results_df['conf_cases'].astype('float')
results_df['prob_cases']=results_df['prob_cases'].astype('float')
results_df['new_case']=results_df['new_case'].astype('float')
results_df['pnew_case']=results_df['pnew_case'].astype('float')
results_df['tot_death']=results_df['tot_death'].astype('float')
results_df['conf_death']=results_df['conf_death'].astype('float')
results_df['prob_death']=results_df['prob_death'].astype('float')
results_df['new_death']=results_df['new_death'].astype('float')
results_df['pnew_death']=results_df['pnew_death'].astype('float')
results_df.dtypes

submission_date     object
state               object
tot_cases          float64
new_case           float64
pnew_case          float64
tot_death          float64
new_death          float64
pnew_death         float64
created_at          object
consent_cases       object
consent_deaths      object
conf_death         float64
prob_death         float64
conf_cases         float64
prob_cases         float64
dtype: object

In [8]:
results_df=results_df.sort_values(by='submission_date')
results_df

,submission_date,state,tot_cases,new_case,pnew_case,tot_death,new_death,pnew_death,created_at,consent_cases,consent_deaths,conf_death,prob_death,conf_cases,prob_cases
9579,2020-01-22,LA,0.0,0.0,NaN,0.0,0.0,NaN,2020-03-26,Not agree,Agree,NaN,NaN,NaN,NaN
19859,2020-01-22,MO,0.0,0.0,NaN,0.0,0.0,NaN,2020-03-26,Not agree,Not agree,NaN,NaN,NaN,NaN
18660,2020-01-22,DE,0.0,0.0,NaN,0.0,0.0,NaN,2020-03-26,Agree,Agree,NaN,NaN,NaN,NaN
1416,2020-01-22,AS,0.0,0.0,NaN,0.0,0.0,NaN,2020-03-26,NaN,NaN,NaN,NaN,NaN,NaN
8096,2020-01-22,WI,0.0,0.0,NaN,0.0,0.0,NaN,2020-03-26,Agree,Agree,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10740,2021-02-04,NM,176211.0,559.0,0.0,3355.0,17.0,0.0,2021-02-05,N/A,Not agree,NaN,NaN,NaN,NaN
2237,2021-02-04,MN,465176.0,1410.0,218.0,6319.0,17.0,0.0,2021-02-05,N/A,Agree,5997.0,322.0,NaN,NaN
9694,2021-02-04,AZ,771796.0,4417.0,651.0,13752.0,176.0,11.0,2021-02-05,Agree,Agree,12224.0,1528.0,722252.0,49544.0
10893,2021-02-04,ND,98095.0,61.0,24.0,1426.0,4.0,0.0,2021-02-05,Agree,Not agree,NaN,NaN,93583.0,4512.0


In [9]:
from sqlalchemy import create_engine

In [10]:
from config import location

In [11]:
engine=create_engine(location)

In [12]:
from sqlalchemy import event

In [13]:
@event.listens_for(engine, "before_cursor_execute")
def receive_before_cursor_excute(
    conn, cursor, statement, params, context, excutemany
    ):
        if excutemany:
            cursor.fast_excutemany=True

In [14]:
receive_before_cursor_excute

<function __main__.receive_before_cursor_excute(conn, cursor, statement, params, context, excutemany)>

In [15]:
#pandas dataframe stored in a postgresql database
results_df.to_sql('covid_19_us_cases',engine,index=False,if_exists="replace",chunksize=100,method='multi')

In [16]:
us_cases_by_date=results_df.groupby(["submission_date"]).sum()
us_cases_by_date

,tot_cases,new_case,pnew_case,tot_death,new_death,pnew_death,conf_death,prob_death,conf_cases,prob_cases
submission_date,,,,,,,,,,
2020-01-22,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-23,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-24,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-25,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-26,5.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2021-01-31,26034475.0,112772.0,14670.0,439955.0,1920.0,388.0,228276.0,27572.0,12054685.0,1575882.0
2021-02-01,26160210.0,125735.0,14793.0,441831.0,1876.0,271.0,229186.0,27756.0,12103533.0,1584363.0
2021-02-02,26277125.0,116915.0,22850.0,445264.0,3433.0,443.0,230661.0,28088.0,12144016.0,1597668.0


In [17]:
#pandas dataframe stored in a postgresql database
us_cases_by_date.to_sql('covid_19_us_cases_by_date',engine,index=True,if_exists="replace",chunksize=100,method='multi')